In [1]:
%cd ../..
%pip install .
import time
# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50, 
#         with viewAngle 135 deg
# ped = Pedestrian on roadsOrIntersections,
#         with regionContainedIn roadRegion,
#         facing Range(-180, 180) deg

# require abs(relative heading of ped from ego) > 70 deg

/home/yongming/workspace/research/apperception_new_local/apperception
Processing /home/yongming/workspace/research/apperception_new_local/apperception
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for apperception: filename=apperception-0.1.0-py3-none-any.whl size=74127 sha256=425ccc5a98219960f042bd3338ccdd98a114ead88b4a6a3399ef0ac95ad54835
  Stored in directory: /tmp/pip-ephem-wheel-cache-1hexmyjn/wheels/a4/33/17/59e20ccef806baa1a68832d19de43f0fe8d640854903fd1772
Successfully built apperception
  Attempting uninstall: apperception
    Found existing installation: apperception 0.1.0
    Uninstalling apperception-0.1.0:
      Successfully uninstalled apperception-0.1.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
"""
with (
  select *
  from item_traj
  where item_traj.object_type = 'pedestrian'
) as pedestrians

select *
from pedestrians as t1
join Cameras as cam on t1.cameraId = Cameras.id
where
  t1.heading < 180 and t1.heading > -180 and
  (contained(t1.traj, road_segment("roads"), cam.timestamp) or
   contained(t1.traj, road_segment("Intersections"), cam.timestamp)) and
  (facingRelative(t1.heading, cam.egoHeading, cam.timestamp) < -70 OR
  facingRelative(t1.heading, cam.egoHeading, cam.timestamp) > 70) AND
  facingRelative(cam.egoHeading, road_direction(cam.ego_translation), cam.timestamp) >= -15 AND
  facingRelative(cam.egoHeading, road_direction(cam.ego_translation), cam.timestamp) <= 15 AND
  DISTANCE(cam.egoTranslation, t1.centroid, cam.timestamp) < 50 AND
  viewAngle(t1.traj, cam.egoHeading, cam.ego_translation, cam.timestamp) < 135
"""

In [3]:
### Prepare the world for queries ###
from apperception.new_world import *
# directly ingest the mini dataset and boston roadnetwork if needed
# from apperception.new_db import database
# database.reset()
# from benchmarks.ingest_scenic_data import ingest_data
# ingest_data()

name = 'ScenicWorld' # world name
world = empty_world(name=name)

### Query ###
from apperception.utils import F
world = world.filter(lambda obj: obj.object_type == 'vehicle.pedestrian')
def pred(obj1, cam):
    return (
        obj1.heading < 180 and obj1.heading > -180 and
        (F.contained(obj1.traj, F.road_segment("road"), cam.timestamp) or
            F.contained(obj1.traj, F.road_segment("intersection"))) and
        (F.facing_relative(obj1, cam.ego, cam.timestamp) < -70 or
            F.facing_relative(obj1, cam.ego, cam.timestamp) >= 70) and
        F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) >= -15 and
        F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) <= 15 and
        F.distance(cam.ego, obj1, cam.timestamp) < 50 and
        F.viewAngle(obj1, cam.ego, cam.timestamp) < 135
    )
world = world.filter("lambda obj1, cam: F.facing_relative(obj1, 0, cam.timestamp) < 180 and " + 
                     "F.facing_relative(obj1, 0, cam.timestamp) > -180 and " +
        "(F.contained(obj1.traj, F.road_segment('road'), cam.timestamp) or " + 
            "F.contained(obj1.traj, F.road_segment('intersection'), cam.timestamp)) and " +
        "(F.facing_relative(obj1, cam.ego, cam.timestamp) < -70 or " + 
            "F.facing_relative(obj1, cam.ego, cam.timestamp) >= 70) and " + 
        "F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) >= -15 and " +
        "F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) <= 15 and " +
        "F.distance(cam.ego, obj1, cam.timestamp) < 50 and " +
        "F.view_angle(obj1, cam.ego, cam.timestamp) < 135")

start = time.time()

key = world.get_traj_key()

end = time.time()
print(format(end-start))

execute: filter
execute: filter
execute: get_traj_key
get_traj_key 
        SELECT itemId FROM (
        SELECT DISTINCT table_0.*
        FROM (
        SELECT DISTINCT table_0.*
        FROM (SELECT * FROM item_general_trajectory) as table_0
        
        JOIN Cameras ON Cameras.cameraId = table_0.cameraId
        WHERE (table_0.objectType='vehicle.pedestrian')
        ) as table_0
        
        JOIN Cameras ON Cameras.cameraId = table_0.cameraId
        WHERE ((facingRelative(table_0.itemHeadings, 0, Cameras.timestamp)<180) AND (facingRelative(table_0.itemHeadings, 0, Cameras.timestamp)>(-180)) AND (contained(table_0.trajCentroids, roadSegment('road'), Cameras.timestamp) OR contained(table_0.trajCentroids, roadSegment('intersection'), Cameras.timestamp)) AND ((facingRelative(table_0.itemHeadings, Cameras.egoHeading, Cameras.timestamp)<(-70)) OR (facingRelative(table_0.itemHeadings, Cameras.egoHeading, Cameras.timestamp)>=70)) AND (distance(Cameras.egoTranslation, table_0.trajC